In [1]:
import numpy as np
import pandas as pd
import os

import time
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import model_selection, preprocessing, metrics, ensemble

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

import lightgbm as lgb

In [2]:
train_df = pd.read_csv("Train.csv")
test_df = pd.read_csv("Test.csv")
submission_df = pd.read_csv("sample_submission.csv")

train_df.head()

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,...,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,EID_23371,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,...,type2,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841
1,EID_18000,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,...,type2,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670
2,EID_3891,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,...,type2,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851
3,EID_17492,F,26.0,3,Single,Lebanon,Human Resource Management,Behavioral,4.0,3,...,type2,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668
4,EID_22534,F,31.0,1,Married,Springfield,Logistics,Conceptual,5.0,4,...,type3,3.0,1,0.7516,-0.4537,2.0,2,8,2,0.1827


In [3]:
train_ids = set(train_df['Employee_ID'].unique())
test_ids = set(test_df['Employee_ID'].unique())

print("Train ids count : ", len(train_ids))
print("Test ids count : ",len(test_ids))
print("Common ids count : ", len(train_ids.intersection(test_ids)))

Train ids count :  7000
Test ids count :  3000
Common ids count :  0


In [4]:
dupes = train_df.duplicated()
sum(dupes)

0

In [5]:
d1=train_df.isnull().sum().to_frame().rename(columns={0: "Train_Null_Values"})
d2=test_df.isnull().sum().to_frame().rename(columns={0: "Test_Null_Values"})
d3=train_df.dtypes.to_frame().rename(columns={0: "Data_Type"})
pd.concat([d1, d2,d3], axis=1)

,Train_Null_Values,Test_Null_Values,Data_Type
Employee_ID,0,0.0,object
Gender,0,0.0,object
Age,412,161.0,float64
Education_Level,0,0.0,int64
Relationship_Status,0,0.0,object
Hometown,0,0.0,object
Unit,0,0.0,object
Decision_skill_possess,0,0.0,object
Time_of_service,144,52.0,float64
Time_since_promotion,0,0.0,int64


In [6]:
missing_cols = [col for col in train_df.columns if train_df[col].isnull().any()]

In [7]:
missing_cols

['Age', 'Time_of_service', 'Pay_Scale', 'Work_Life_balance', 'VAR2', 'VAR4']

In [8]:
train_df.columns

Index(['Employee_ID', 'Gender', 'Age', 'Education_Level',
       'Relationship_Status', 'Hometown', 'Unit', 'Decision_skill_possess',
       'Time_of_service', 'Time_since_promotion', 'growth_rate', 'Travel_Rate',
       'Post_Level', 'Pay_Scale', 'Compensation_and_Benefits',
       'Work_Life_balance', 'VAR1', 'VAR2', 'VAR3', 'VAR4', 'VAR5', 'VAR6',
       'VAR7', 'Attrition_rate'],
      dtype='object')

In [9]:
cols = train_df.columns

NUM = (train_df._get_numeric_data().columns).to_list()
CAT= cols.difference(NUM).to_list()

In [10]:
NUM

['Age',
 'Education_Level',
 'Time_of_service',
 'Time_since_promotion',
 'growth_rate',
 'Travel_Rate',
 'Post_Level',
 'Pay_Scale',
 'Work_Life_balance',
 'VAR1',
 'VAR2',
 'VAR3',
 'VAR4',
 'VAR5',
 'VAR6',
 'VAR7',
 'Attrition_rate']

In [11]:
CAT

['Compensation_and_Benefits',
 'Decision_skill_possess',
 'Employee_ID',
 'Gender',
 'Hometown',
 'Relationship_Status',
 'Unit']

In [12]:
cat_cols = CAT

nunique_dict = {}
for c in cat_cols:
    nunique_dict[c] = train_df[c].nunique()
    print(c + ': %d values' % nunique_dict[c])

Compensation_and_Benefits: 5 values
Decision_skill_possess: 4 values
Employee_ID: 7000 values
Gender: 2 values
Hometown: 5 values
Relationship_Status: 2 values
Unit: 12 values


In [13]:
cat_unique=[]
for cat in CAT:
    a=pd.unique(train_df[cat])
    if len(a)>1 and len(a)<500:
        cat_unique.append(cat)
cat_unique 

['Compensation_and_Benefits',
 'Decision_skill_possess',
 'Gender',
 'Hometown',
 'Relationship_Status',
 'Unit']

In [14]:
for cat in cat_unique:
    x=pd.value_counts(train_df[cat])
    x=pd.DataFrame(x)
    x['percentage']=x/len(train_df)
    x['var']=cat
    x['category']=x.index.values
    x.columns=['count','percentage','var','category']
    if cat==cat_unique[0]:
        freq_table=x
    else:
        freq_table=x.append([freq_table])
freq_table

,count,percentage,var,category
IT,1394,0.199143,Unit,IT
Logistics,1173,0.167571,Unit,Logistics
Sales,943,0.134714,Unit,Sales
Operarions,706,0.100857,Unit,Operarions
R&D,680,0.097143,Unit,R&D
Purchasing,504,0.072000,Unit,Purchasing
Accounting and Finance,496,0.070857,Unit,Accounting and Finance
Human Resource Management,344,0.049143,Unit,Human Resource Management
Marketing,212,0.030286,Unit,Marketing
Production,211,0.030143,Unit,Production


In [15]:
num_cols=['Age',
 'Education_Level',
 'Time_of_service',
 'Time_since_promotion',
 'growth_rate',
 'Travel_Rate',
 'Post_Level',
 'Pay_Scale',
 'Work_Life_balance',
 'VAR1',
 'VAR2',
 'VAR3',
 'VAR4',
 'VAR5',
 'VAR6',
 'VAR7']

In [16]:
missing_values=['VAR4','VAR2','Age','Time_of_service','Work_Life_balance','Pay_Scale']

In [17]:
cat_unique

['Compensation_and_Benefits',
 'Decision_skill_possess',
 'Gender',
 'Hometown',
 'Relationship_Status',
 'Unit']